In [ ]:
from ipywidgets import interact
from paraview.simple import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
print("Using ", GetParaViewSourceVersion())
#### disable automatic camera reset on 'Show'
paraview.simple._DisableFirstRenderCameraReset()

def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(disp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

In [ ]:
# create three orthogonal planes
plane3 = Plane()
plane3.Origin = [-1.0, -1.0, -1.0]
plane3.Point1 = [1.0, -1.0, -1.0]
plane3.Point2 = [-1.0, 1.0, -1.0]
plane3.XResolution = 10
plane3.YResolution = 10
p3 = Show()

plane1 = Plane()
plane1.Origin = [-1.0, -1.0, -1.0]
plane1.Point1 = [1.0, -1.0, -1.0]
plane1.Point2 = [-1.0, -1.0, 1.0]
plane1.XResolution = 10
plane1.YResolution = 10
p1 = Show()

plane2 = Plane()
plane2.Origin = [-1.0, -1.0, -1.0]
plane2.Point1 = [-1.0, -1.0, 1.0]
plane2.Point2 = [-1.0, 1.0, -1.0]
plane2.XResolution = 10
plane2.YResolution = 10
p2=Show()

# create a Torus
superquadric1 = Superquadric()
superquadric1.ThetaResolution = 64
superquadric1.PhiResolution = 64

torus=Show()

In [ ]:
# Create a new 'Render View'
renderView1 = GetRenderView()
renderView1.OrientationAxesVisibility = 0

renderView1.EnableRayTracing = 0 # Classic OpenGL

In [ ]:
renderView1.ViewSize = [400, 400]

In [ ]:
if GetParaViewSourceVersion() >= "paraview version 5.8.0":
    from paraview.modules.vtkRemotingViews import vtkPVOpenGLInformation
else:
    from paraview.modules.vtkPVClientServerCoreRendering import vtkPVOpenGLInformation

info = vtkPVOpenGLInformation()
info.CopyFromObject(None)
print("Vendor:   %s" % info.GetVendor())
print("Version:  %s" % info.GetVersion())
print("Renderer: %s" % info.GetRenderer())

In [ ]:
# import the PVDisplay widget, then instantiate one for renV

from ipyparaview.widgets import PVDisplay
disp = PVDisplay(renderView1)

# show the widget once
display(disp)

In [ ]:
# Switch to Flat Shading
torus.Interpolation = 'Flat'
torus.Specular = 0

In [ ]:
# Switch to Gouraud Shading
torus.Interpolation = 'Gouraud'
torus.Specular = 0.5

In [ ]:
# Switch to PBR Shading
torus.Interpolation = 'PBR'
torus.Metallic = .5
# how glossy is the material? glossy materials have a low roughness value
torus.Roughness = 0.9
torus.AmbientColor = [1.0, 0.666, 0.5]
torus.DiffuseColor = [1.0, 0.666, 0.5]

def ChangeRoughnessMetallicity(Roughness, Metallicity):
    torus.Roughness = Roughness
    torus.Metallic = Metallicity
    
interact(ChangeRoughnessMetallicity, Roughness=(0.,1.), Metallicity=(0.,1.))

# Image-Based Lighting

Glossy materials (low roughness value) are reflecting the environment with a technique called Image-Based Lighting that is supported by the PBR shading model

The environment is made of a cubemap texture which is a texture consisting of 6 seamless images for the 6 different directions of the 3D space

In [ ]:
# Load a cubemap texture. https://hdrihaven.com/hdris/ has lots of examples
imgFileName = "../Data/palermo_sidewalk_2k.jpg"
img = mpimg.imread(imgFileName)
fig, ax = plt.subplots(figsize=(8,4))
imgplot = ax.imshow(img)

In [ ]:
# make the material glossy and hide the three planes for a moment
torus.Roughness = 0.
Hide(plane1)
Hide(plane2)
Hide(plane3)

cubemap = CreateTexture(imgFileName)
renderView1.UseSkyboxBackground = 1
renderView1.UseEnvironmentLighting = 1
renderView1.BackgroundTexture = cubemap